# PageRank [39 points]

In this notebook, you'll implement the [PageRank algorithm](http://ilpubs.stanford.edu:8090/422/) summarized in class. You'll test it on a real dataset (circa 2005) that consists of [political blogs](http://networkdata.ics.uci.edu/data/polblogs/) and their links among one another.

For today's notebook, you'll need to download the following additional materials:
* A SQLite version of the political blogs dataset: http://cse6040.gatech.edu/datasets/poliblogs.db (~ 611 KiB)

In [5]:
# Some modules you'll need
from IPython.display import display
import numpy as np
import scipy.sparse as sp
import scipy.io as spio
import cse6040utils

## Part 1: Explore the Dataset

Let's start by looking at the dataset, to get a feel for what it contains.

In this part, try to rely primarily on SQL queries to accomplish each task. This scenario is appropriate if the database is so large that you cannot expect to load it all into memory.

Incidentally, one of you asked recently how to get the schema for a SQLite database when using Python. Here is some code adapted from a few ideas floating around on the web. Let's use these to inspect the tables available in the political blogs dataset.

In [7]:
import sqlite3 as db
import pandas as pd

def get_table_names (conn):
    assert type (conn) == db.Connection # Only works for sqlite3 DBs
    query = "select name from sqlite_master where type='table'"
    return pd.read_sql_query (query, conn)

def print_schemas (conn, table_names=None, limit=0):
    assert type (conn) == db.Connection # Only works for sqlite3 DBs
    if table_names is None:
        table_names = get_table_names (conn)
    c = conn.cursor ()
    query = "pragma table_info ({table})"
    for name in table_names:
        c.execute (query.format (table=name))
        columns = c.fetchall ()
        print ("=== {table} ===".format (table=name))
        col_string = "[{id}] {name} : {type}"
        for col in columns:
            print (col_string.format (id=col[0],
                                      name=col[1],
                                      type=col[2]))
        print ("\n")

In [8]:
conn = db.connect ('poliblogs.db')

for name in get_table_names (conn)['name']:
    print_schemas (conn, [name])
    query = '''select * from %s limit 5''' % name #string substitution
    display (pd.read_sql_query (query, conn))
    print ("\n")

=== Vertices ===
[0] index : INTEGER
[1] Id : INTEGER
[2] Url : TEXT
[3] Leaning : TEXT




,index,Id,Url,Leaning
0,0,1,100monkeystyping.com,Left
1,1,2,12thharmonic.com/wordpress,Left
2,2,3,40ozblog.blogspot.com,Left
3,3,4,4lina.tblog.com,Left
4,4,5,750volts.blogspot.com,Left




=== Edges ===
[0] index : INTEGER
[1] Source : INTEGER
[2] Target : INTEGER




,index,Source,Target
0,0,267,1394
1,1,267,483
2,2,267,1051
3,3,904,1479
4,4,904,919


**Exercise 1.** (3 points). Write a snippet of code to verify that the vertex IDs are _dense_ in some interval $[1, n]$. That is, there is a minimum value of $1$, some maximum value $n$, and _no_ missing values between $1$ and $n$.

Also store the number of vertices $n$ in a variable named `num_vertices`.

In [9]:
query = '''
    select min(Id) , max(Id), count(distinct Id)
    from Vertices
'''

df = pd.read_sql_query(query, conn)

num_vertices = df['count(distinct Id)'][0]



In [10]:
assert num_vertices == 1490

**Exercise 2** (3 points). Make sure every edge has its end points in the vertex table.

In [11]:
query = '''  
    
    select Source
    from Edges
    union 
    select Target
    from Edges
    except
    select Id
    from Vertices
'''

pd.read_sql_query(query, conn)

,Source


In [12]:
query = '''
  select min (Source), max (Source), min (Target), max (Target) from Edges
'''
pd.read_sql_query (query, conn)

,min (Source),max (Source),min (Target),max (Target)
0,1,1490,1,1489


**Exercise 3** (2 points). Determine which vertices have no outgoing edges. Store the result in a Pandas `DataFrame` named `df_deadends` with a single column named `Id`.

In [15]:
query = '''
select Id 
from Vertices a
left join
Edges b
on a.id = b.Source 
where b.Source is null
'''

df_deadends = pd.read_sql_query(query, conn)

display (df_deadends.head ())
display (df_deadends.tail ())

,Id
0,3
1,4
2,7
3,25
4,30


,Id
420,1467
421,1470
422,1474
423,1480
424,1483


In [16]:
print ("\n==> %d vertices have no outgoing edges." % len (df_deadends))

df_deadends_soln = pd.read_csv ('df_deadends_soln.csv')
assert cse6040utils.tibbles_are_equivalent (df_deadends, df_deadends_soln)

print ("\n(Passed.)")


==> 425 vertices have no outgoing edges.

(Passed.)


**Exercise 4** (2 points). Determine which vertices that have no incoming edges. Store the result in a Pandas `DataFrame` called `df_nolove` having just a single column named `Id` to hold the corresponding vertex IDs.

In [17]:
# YOUR CODE HERE
query = '''
select Id from
Vertices a
left join 
Edges b
on a.Id = b.Target
where b.Target is null

'''

df_nolove = pd.read_sql_query(query, conn)



display (df_nolove.head ())
display (df_nolove.tail ())

,Id
0,3
1,4
2,6
3,9
4,11


,Id
495,1481
496,1483
497,1484
498,1488
499,1490


In [18]:
print ("\n==> %d vertices have no incoming edges." % len (df_nolove))

df_nolove_soln = pd.read_csv ('df_nolove_soln.csv')
assert cse6040utils.tibbles_are_equivalent (df_nolove, df_nolove_soln)

print ("\n(Passed.)")


==> 500 vertices have no incoming edges.

(Passed.)


**Exercise 5** (3 points). Compute an [SQL view](https://www.sqlite.org/lang_createview.html) called `Outdegrees`, which contains the following columns:

1. `Id`: vertex ID
2. `Degree`: the out-degree of the corresponding vertex.

To help you check your view, the test code selects from your view but adds a `Url` and `Leaning` fields, ordering the results in descending order of degree. It also prints first few and last few rows of this query, so you can inspect the URLs as a sanity check. (Perhaps it also provides a small bit of entertainment!)

In [19]:
# Remove an existing view, if it exists:
c = conn.cursor ()
c.execute ('drop view if exists Outdegrees')

query = '''
    create view Outdegrees as
    select Source as Id, count(*) as Degree
    from Edges
    group by Source
'''
c.execute(query)


In [20]:
query = '''
  select Outdegrees.Id, Degree, Url, Leaning
    from Outdegrees, Vertices
    where Outdegrees.Id=Vertices.Id
    order by -Degree
'''
df_outdegrees = pd.read_sql_query (query, conn)
print ("==> A few entries with large out-degrees:")
display (df_outdegrees.head (10))
print ("\n==> A few entries with small out-degrees:")
display (df_outdegrees.tail ())

df_outdegrees_soln = pd.read_csv ('outdegrees_soln.csv')
assert cse6040utils.tibbles_are_equivalent (df_outdegrees, df_outdegrees_soln)

print ("\n(Passed.)")

==> A few entries with large out-degrees:


,Id,Degree,Url,Leaning
0,855,256,blogsforbush.com,Right
1,454,140,newleftblogs.blogspot.com,Left
2,387,131,madkane.com/notable.html,Left
3,512,131,politicalstrategy.org,Left
4,880,123,cayankee.blogs.com,Right
5,363,115,liberaloasis.com,Left
6,1101,113,lashawnbarber.com,Right
7,1000,110,gevkaffeegal.typepad.com/the_alliance,Right
8,524,109,presidentboxer.blogspot.com,Left
9,144,106,corrente.blogspot.com,Left



==> A few entries with small out-degrees:


,Id,Degree,Url,Leaning
1060,1468,1,watchandwait.blogspot.com,Right
1061,1481,1,writehouse.us,Right
1062,1486,1,youngconservative.blogspot.com,Right
1063,1488,1,zeke01.blogspot.com,Right
1064,1490,1,zeph1z.tripod.com/blog,Right



(Passed.)


**Exercise 6** (3 points). Compute an [SQL view](https://www.sqlite.org/lang_createview.html) called `Indegrees`, which contains the following columns:

1. `Id`: vertex ID
2. `Degree`: the in-degree of this vertex.

Your view should only include vertices with positive out-degree. (That is, if the in-degree is zero, you may leave it out of the resulting view.)

In [23]:
# Remove an existing view, if it exists:
c = conn.cursor ()
c.execute ('drop view if exists Indegrees')


# YOUR CODE HERE

query  = '''
create view Indegrees as 

    select Target as Id, count(*) as Degree
    from Edges
    group by Target
'''

c.execute(query)

In [24]:
query = '''
  select Indegrees.Id, Degree, Url, Leaning
    from Indegrees, Vertices
    where Indegrees.Id=Vertices.Id
    order by -Degree
'''
df_outdegrees = pd.read_sql_query (query, conn)
print ("==> A few entries with large in-degrees:")
display (df_outdegrees.head (10))
print ("\n==> A few entries with small in-degrees:")
display (df_outdegrees.tail ())

df_outdegrees_soln = pd.read_csv ('indegrees_soln.csv')
assert cse6040utils.tibbles_are_equivalent (df_outdegrees, df_outdegrees_soln)

print ("\n(Passed.)")

==> A few entries with large in-degrees:


,Id,Degree,Url,Leaning
0,155,338,dailykos.com,Left
1,1051,277,instapundit.com,Right
2,641,269,talkingpointsmemo.com,Left
3,55,264,atrios.blogspot.com,Left
4,963,240,drudgereport.com,Right
5,1245,221,powerlineblog.com,Right
6,855,212,blogsforbush.com,Right
7,729,201,washingtonmonthly.com,Left
8,1153,201,michellemalkin.com,Right
9,1437,187,truthlaidbear.com,Right



==> A few entries with small in-degrees:


,Id,Degree,Url,Leaning
985,1451,1,usdlaw.blogspot.com,Right
986,1467,1,warriorscreed.000k3.com,Right
987,1473,1,wholewheatblogger.com,Right
988,1474,1,wildweasel.blog-city.com,Right
989,1485,1,xtremerightwing.net,Right



(Passed.)


**Exercise 7** (5 points). Query the database to extract a report of which URLs point to which URLs, storing the result in a Pandas data frame called `df_G`. This data frame should have these columns:

- `SourceURL`: URL of a source vertex
- `SourceLeaning`: "Leaning" value of that vertex
- `TargetURL`: URL of the corresponding target vertex
- `TargetLeaning`: "Leaning" value of that vertex

In [26]:
query = '''
select b.Url as SourceURL
    , b.Leaning as SourceLeaning
    , c.Url as TargetURL
    , c.Leaning as TargetLeaning
    
from Edges a
inner join
Vertices b
on a.Source = b.Id
inner join
Vertices c
on a.Target = c.id

'''


df_G = pd.read_sql_query(query, conn)

In [27]:
from IPython.display import display
display (df_G.head ())
print ("...")
display (df_G.tail ())

df_G_soln = pd.read_csv ('df_G_soln.csv')
assert cse6040utils.tibbles_are_equivalent (df_G, df_G_soln)

,SourceURL,SourceLeaning,TargetURL,TargetLeaning
0,home.earthlink.net/~kevin.omeara,Left,thecommonvirtue.com,Right
1,home.earthlink.net/~kevin.omeara,Left,oliverwillis.com,Left
2,home.earthlink.net/~kevin.omeara,Left,instapundit.com,Right
3,conservativecat.com,Right,wizbangblog.com,Right
4,conservativecat.com,Right,coxandforkum.com,Right


...


,SourceURL,SourceLeaning,TargetURL,TargetLeaning
19085,marknicodemo.blogspot.com,Right,thatliberalmedia.com,Right
19086,marknicodemo.blogspot.com,Right,timblair.net,Right
19087,marknicodemo.blogspot.com,Right,thevaticanofliberalism.com,Right
19088,marknicodemo.blogspot.com,Right,thepatriette.com,Right
19089,marknicodemo.blogspot.com,Right,michaeltotten.com,Right


## Part 2: Implement PageRank

The following exercises will walk you through a possible implementation of PageRank for this dataset.

**Exercise 8** (5 points). Build a sparse matrix, `PT`, as a Scipy CSR matrix that stores $P^T \equiv G^TD^{-1}$, where $G^T$ is the transpose of the connectivity matrix $G$, and $D^{-1}$ is the diagonal matrix of inverse out-degrees. Be sure also to do the following:

1. Recall that the database indices are 1-based; when converting to the Scipy representation, you should convert these to be 0-based.
2. To ensure that there is no "information loss," place a 1.0 at any diagonal entry where there are no outgoing edges.
3. Ensure that `PT` is square with dimension `num_vertices`.

In [120]:
#Edges: Source, Target
#Outdegrees: Id, Degree
query = '''
    select a.* from 
    (select Source as Col, Target as Row, 1.0/Degree as Value
    from Edges, Outdegrees
    where Source = Id
    order by Col desc) a   
    
    union all
    
    select b.id, b.id, 1.0
    from
    (select distinct id
    from Vertices
    except 
    select Source from Edges 
     )b
    
'''

df = pd.read_sql_query(query, conn)
df.head(10)

import scipy.sparse as sp
columns = [i - 1 for i in df.iloc[:, 0]]
rows =    [i - 1 for i in df.iloc[:, 1]]

df_coo_sp = sp.coo_matrix((df['Value'], (rows, columns))
                          , shape = (len(df["Col"].unique()) , len(df["Col"].unique()) )
                         )

PT = df_coo_sp.tocsr ()




In [117]:
assert PT.shape == (num_vertices, num_vertices)
assert len (PT.indices) == 19450

# Check that columns sum to 1.0
u = np.ones (num_vertices)
y_u = np.transpose (PT).dot (u)
assert (np.max (np.abs (y_u - u))) <= 3e-15

# Check whether `PT` matches what we expect
#assert (spio.loadmat ('PT.mat')['PT'] != PT).getnnz () == 0

print ("\n(Passed.)")


(Passed.)


**Exercise 9** (10 points). Complete the PageRank implementation for this dataset. To keep it simple, you may take $\alpha=0.85$, $x(0)$ equal to the vector of all $1/n$ values, and 25 iterations.

> **Note.** This implementation asks you to maintain a list, `X`, that stores every `x(t)` that you compute in sequence.

In [118]:
# YOUR CODE GOES BELOW. We've provided some scaffolding code,
# so you just need to complete it.

ALPHA = 0.85 # Probability of following some link
MAX_ITERS = 25
n = num_vertices

# Let X[t] store the dense vector x(t) at time t
X = []

x_0 = np.ones (n) / n # Initial distribution: 1/n at each page
X.append (x_0)

for t in range (1, MAX_ITERS):
    # Complete this implementation
    
    X.append(ALPHA * PT.dot(X[t-1]) + ((1-ALPHA)/n ) * (np.ones(n)))

In [119]:
# Write some code here to create a table in the database
# called PageRank

command = '''DROP TABLE IF EXISTS PageRank'''
c = conn.cursor ()
c.execute (command)

command = '''CREATE TABLE PageRank (Id INTEGER, Rank REAL)'''
c.execute (command)

command = '''INSERT INTO PageRank VALUES (?, ?)'''
c.executemany (command, zip (range (1, n+1), X[-1]))

# Complete this query:
query = '''
  SELECT Rank, V.Id, I.Degree AS InDegree, O.Degree AS OutDegree, V.Url, V.Leaning
    FROM PageRank AS P, Vertices AS V, Indegrees AS I, Outdegrees AS O
    WHERE (P.Id = V.Id) AND (P.Id = I.Id) AND (P.Id = O.Id)
    ORDER BY -Rank
    LIMIT 10
'''
df_ranks = pd.read_sql_query (query, conn)
display (df_ranks)

assert df_ranks['Url'][0] == 'dailykos.com'
assert df_ranks['Url'][1] == 'atrios.blogspot.com'
assert df_ranks['Url'][2] == 'instapundit.com'
assert df_ranks['Url'][3] == 'blogsforbush.com'
assert df_ranks['Url'][4] == 'talkingpointsmemo.com'

print ("\n(Passed.)")

,Rank,Id,InDegree,OutDegree,Url,Leaning
0,0.009634,155,338,46,dailykos.com,Left
1,0.008179,55,264,87,atrios.blogspot.com,Left
2,0.006780,1051,277,86,instapundit.com,Right
3,0.006705,855,212,256,blogsforbush.com,Right
4,0.006678,641,269,14,talkingpointsmemo.com,Left
5,0.005859,1153,201,28,michellemalkin.com,Right
6,0.005748,963,240,5,drudgereport.com,Right
7,0.005665,729,201,55,washingtonmonthly.com,Left
8,0.004798,1245,221,15,powerlineblog.com,Right
9,0.004572,323,165,9,juancole.com,Left



(Passed.)


**Exercise 10** (3 points). The `Vertices` table includes a column called, `Leaning`, which expresses a political leaning -- either "Left" or "Right". How might you use this column to come up with an alternative ranking scheme?

ANSWER:

We could use the Leaning column to modify the PageRanks further, i.e. either make the weight of the edges (transitions) between the same Leaning values higher, or assume that there will not be transitions between pages from different Leaning values.